In [ ]:
import tensorflow as tf

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist

In [ ]:
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]

In [ ]:
X_train, X_valid, X_test = X_train / 255., X_valid / 255., X_test/ 255.

In [ ]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress",
"Coat",
"Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [ ]:
tf.random.set_seed(42)
model = tf.keras.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=[28, 28]))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(300, activation="relu"))
model.add(tf.keras.layers.Dense(100, activation="relu"))
model.add(tf.keras.layers.Dense(10, activation="softmax"))

In [ ]:
model.summary()

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
optimizer="adam",
metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_valid, y_valid))

In [ ]:
model.evaluate(X_test, y_test)

## `Fine-Tune Hyperparameters`

In [ ]:
%pip install keras-tuner

In [ ]:
import keras_tuner as kt

def build_model(hp):
  n_hidden = hp.Int("n_hidden", min_value=0, max_value=8,
  default=2)
  n_neurons = hp.Int("n_neurons", min_value=16, max_value=256)
  learning_rate = hp.Float("learning_rate", min_value=1e-4,max_value=1e-2,
  sampling="log")
  optimizer = hp.Choice("optimizer", values=["sgd", "adam"])
  if optimizer == "sgd":
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
  else:
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Flatten())
  for _ in range(n_hidden):
    model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))

  model.add(tf.keras.layers.Dense(10, activation="softmax"))
  model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, 
                metrics=["accuracy"])
  return model

In [ ]:
class MyClassificationHyperModel(kt.HyperModel):
  def build(self, hp):
    return build_model(hp)
  def fit(self, hp, model, X, y, **kwargs):
    batch_size = hp.Int("batch_size", 32, 128, step=32, default=64)
  
    return model.fit(X, y, batch_size, **kwargs)

In [ ]:
hb_search_tuner = kt.Hyperband(MyClassificationHyperModel(), 
                                   factor=3, hyperband_iterations=2, 
                                   overwrite=True, directory="my_fashion_mnist",
                                   project_name="hyperband",
                                   max_epochs=10, objective="val_accuracy", 
                                   seed=42)
hb_search_tuner.search(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

In [ ]:
bayesian_opt_tuner = kt.BayesianOptimization(
                                MyClassificationHyperModel(), objective="val_accuracy",
                                seed=42,
                                max_trials=10, alpha=1e-4, beta=2.6,
                                overwrite=True, directory="my_fashion_mnist",
                                project_name="bayesian_opt")
bayesian_opt_tuner.search(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

### *Functional* API, Bouston house

In [ ]:
boston_house = tf.keras.datasets.boston_housing.load_data()

In [ ]:
(X_train, y_train), (X_test, y_test) = boston_house

In [ ]:
X_train.shape

In [ ]:
tf.random.set_seed(42)

norm_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:])
hidden_layer1 = tf.keras.layers.Dense(50, activation="relu")
hidden_layer2 = tf.keras.layers.Dense(50, activation="relu")
hidden_layer3 = tf.keras.layers.Dense(50, activation="relu")
output_layer = tf.keras.layers.Dense(1)

input_ = tf.keras.layers.Input(shape=X_train.shape[1:])
normalized = norm_layer(input_)
hidden1 = hidden_layer1(normalized)
hidden2 = hidden_layer2(hidden1)
hidden3 = hidden_layer2(hidden2)
output = output_layer(hidden3)

model = tf.keras.Model(inputs=[input_], outputs=[output])

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])
norm_layer.adapt(X_train)
history = model.fit(X_train, y_train, validation_split=0.1 ,epochs=20)
mse_test, rmse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3]
y_pred = model.predict(X_new)

### *Subclassing* API, Bouston house 

In [ ]:
class WideAndDeepModel(tf.keras.Model):
  def __init__(self, units=30, activation="relu", **kwargs):
    super().__init__(**kwargs) # needed to support naming the model
    self.norm_layer_wide = tf.keras.layers.Normalization()
    self.norm_layer_deep = tf.keras.layers.Normalization()
    self.hidden1 = tf.keras.layers.Dense(units, activation=activation)
    self.hidden2 = tf.keras.layers.Dense(units, activation=activation)
    self.main_output = tf.keras.layers.Dense(1)
    self.aux_output = tf.keras.layers.Dense(1)
  def call(self, inputs):
    input_wide, input_deep = inputs
    norm_wide = self.norm_layer_wide(input_wide)
    norm_deep = self.norm_layer_deep(input_deep)
    hidden1 = self.hidden1(norm_deep)
    hidden2 = self.hidden2(hidden1)
    concat = tf.keras.layers.concatenate([norm_wide, hidden2])
    output = self.main_output(concat)
    aux_output = self.aux_output(hidden2)
    return output, aux_output

In [ ]:
class PrintValTrainRatioCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    print(logs)
    ratio = logs["val_loss"] / logs["loss"]
    print(f"Epoch={epoch}, val/train={ratio:.2f}")

In [ ]:
test_callback = PrintValTrainRatioCallback()

In [ ]:
model = WideAndDeepModel()

In [ ]:
X_train_wide, X_train_deep = X_train[:, :5], X_train[:, 2:]
model.norm_layer_wide.adapt(X_train_wide)
model.norm_layer_deep.adapt(X_train_deep)
model.compile(loss="mse", optimizer="Adam", metrics=["RootMeanSquaredError"])
history = model.fit([X_train_wide, X_train_deep], y_train, validation_split=0.1 
                    ,epochs=20, callbacks=[test_callback])
X_test_wide, X_test_deep = X_test[:, :5], X_test[:, 2:]
X_new_wide, X_new_deep = X_test_wide[:3], X_test_deep[:3]
rmse_test = model.evaluate((X_test_wide, X_test_deep), y_test)
X_new = X_test[:3]
y_pred = model.predict((X_new_wide, X_new_deep))